## Admission to Grad School - Decission Tree
Using admission data from https://stats.idre.ucla.edu/r/dae/logit-regression/ this notebook modeles the admission using a Decision Tree in Spark. The data is ziped with the notebook.

First we read the data set and divide into training and test sets.

In [5]:
import org.apache.spark.sql.types.{StructField, StructType, StringType, DoubleType, IntegerType}
import org.apache.spark.ml.feature.RFormula

val schema = new StructType(Array(
  new StructField("admit", IntegerType, true),
  new StructField("gre", IntegerType, true),
  new StructField("gpa", DoubleType, true),
  new StructField("rank", IntegerType, true)))

val data = spark.read.format("csv").schema(schema).option("header",true).load("admit_data.csv")

val supervised = new RFormula().setFormula("admit ~ gre + gpa + rank")

val fittedRF = supervised.fit(data)
val preparedDF = fittedRF.transform(data)
val Array(train, test) = preparedDF.randomSplit(Array(0.7, 0.3))
train.show

+-----+---+----+----+----------------+-----+
|admit|gre| gpa|rank|        features|label|
+-----+---+----+----+----------------+-----+
|    0|220|2.83|   3|[220.0,2.83,3.0]|  0.0|
|    0|300|2.92|   4|[300.0,2.92,4.0]|  0.0|
|    0|300|3.01|   3|[300.0,3.01,3.0]|  0.0|
|    0|340| 2.9|   1| [340.0,2.9,1.0]|  0.0|
|    0|360|2.56|   3|[360.0,2.56,3.0]|  0.0|
|    0|360| 3.0|   3| [360.0,3.0,3.0]|  0.0|
|    0|360|3.27|   3|[360.0,3.27,3.0]|  0.0|
|    0|380|2.91|   4|[380.0,2.91,4.0]|  0.0|
|    0|380|3.33|   4|[380.0,3.33,4.0]|  0.0|
|    0|380|3.34|   3|[380.0,3.34,3.0]|  0.0|
|    0|380|3.38|   2|[380.0,3.38,2.0]|  0.0|
|    0|380|3.43|   3|[380.0,3.43,3.0]|  0.0|
|    0|400|3.31|   3|[400.0,3.31,3.0]|  0.0|
|    0|400|3.35|   3|[400.0,3.35,3.0]|  0.0|
|    0|400|3.38|   2|[400.0,3.38,2.0]|  0.0|
|    0|400|3.65|   2|[400.0,3.65,2.0]|  0.0|
|    0|420|2.26|   4|[420.0,2.26,4.0]|  0.0|
|    0|420|2.67|   3|[420.0,2.67,3.0]|  0.0|
|    0|420|2.69|   2|[420.0,2.69,2.0]|  0.0|
|    0|420

In [6]:
test.show

+-----+---+----+----+----------------+-----+
|admit|gre| gpa|rank|        features|label|
+-----+---+----+----+----------------+-----+
|    0|340|2.92|   3|[340.0,2.92,3.0]|  0.0|
|    0|340|3.15|   3|[340.0,3.15,3.0]|  0.0|
|    0|360|3.14|   1|[360.0,3.14,1.0]|  0.0|
|    0|380|2.94|   3|[380.0,2.94,3.0]|  0.0|
|    0|380|3.59|   4|[380.0,3.59,4.0]|  0.0|
|    0|380|3.61|   3|[380.0,3.61,3.0]|  0.0|
|    0|400|2.93|   3|[400.0,2.93,3.0]|  0.0|
|    0|400|3.05|   2|[400.0,3.05,2.0]|  0.0|
|    0|400|3.08|   2|[400.0,3.08,2.0]|  0.0|
|    0|400|3.36|   2|[400.0,3.36,2.0]|  0.0|
|    0|400|3.51|   3|[400.0,3.51,3.0]|  0.0|
|    0|420|3.02|   1|[420.0,3.02,1.0]|  0.0|
|    0|460|3.07|   2|[460.0,3.07,2.0]|  0.0|
|    0|460|3.44|   2|[460.0,3.44,2.0]|  0.0|
|    0|460|3.58|   2|[460.0,3.58,2.0]|  0.0|
|    0|480|3.39|   4|[480.0,3.39,4.0]|  0.0|
|    0|480|3.44|   3|[480.0,3.44,3.0]|  0.0|
|    0|480|3.45|   2|[480.0,3.45,2.0]|  0.0|
|    0|500|2.93|   4|[500.0,2.93,4.0]|  0.0|
|    0|500

Now fit the model to the data and then we can apply the model to the test set.

In [8]:
import org.apache.spark.ml.regression.DecisionTreeRegressionModel
import org.apache.spark.ml.regression.DecisionTreeRegressor

val dt = new DecisionTreeRegressor().setLabelCol("label").setFeaturesCol("features")

val model = dt.fit(train)

val predictions = model.transform(test)

predictions.show

+-----+---+----+----+----------------+-----+-------------------+
|admit|gre| gpa|rank|        features|label|         prediction|
+-----+---+----+----+----------------+-----+-------------------+
|    0|340|2.92|   3|[340.0,2.92,3.0]|  0.0|                0.0|
|    0|340|3.15|   3|[340.0,3.15,3.0]|  0.0|                0.0|
|    0|360|3.14|   1|[360.0,3.14,1.0]|  0.0| 0.3392857142857143|
|    0|380|2.94|   3|[380.0,2.94,3.0]|  0.0|                0.0|
|    0|380|3.59|   4|[380.0,3.59,4.0]|  0.0|0.23076923076923078|
|    0|380|3.61|   3|[380.0,3.61,3.0]|  0.0|0.23076923076923078|
|    0|400|2.93|   3|[400.0,2.93,3.0]|  0.0|                0.0|
|    0|400|3.05|   2|[400.0,3.05,2.0]|  0.0| 0.3392857142857143|
|    0|400|3.08|   2|[400.0,3.08,2.0]|  0.0| 0.3392857142857143|
|    0|400|3.36|   2|[400.0,3.36,2.0]|  0.0| 0.3392857142857143|
|    0|400|3.51|   3|[400.0,3.51,3.0]|  0.0|0.23076923076923078|
|    0|420|3.02|   1|[420.0,3.02,1.0]|  0.0| 0.3392857142857143|
|    0|460|3.07|   2|[460

Some information about the model. First root mean square error (rmse) and then the decision tree. On the rmse wikipedia states: 
> Although RMSE is one of the most commonly reported measures of disagreement, some scientists misinterpret RMSD as average error, which RMSD is not. RMSD is the square root of the average of squared errors, thus RMSD confounds information concerning average error with information concerning variation in the errors. The effect of each error on RMSD is proportional to the size of the squared error thus larger errors have a disproportionately large effect on RMSD. Consequently, RMSD is sensitive to outliers

The decision tree has 47 node. While it almost impossible to match the else's with the ifs it does give you a feel for how the tree makes its decisons.

In [10]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

val evaluator = new RegressionEvaluator().
  setLabelCol("label").
  setPredictionCol("prediction").
  setMetricName("rmse")
  
val rmse = evaluator.evaluate(predictions)
println("Root Mean Squared Error (RMSE) on test data = " + rmse)

println("Learned regression tree model:\n" + model.toDebugString)

Root Mean Squared Error (RMSE) on test data = 0.45132128346081307
Learned regression tree model:
DecisionTreeRegressionModel (uid=dtr_3b434a5e78b0) of depth 5 with 47 nodes
  If (feature 1 <= 3.43)
   If (feature 2 <= 2.0)
    If (feature 0 <= 740.0)
     If (feature 0 <= 300.0)
      Predict: 1.0
     Else (feature 0 > 300.0)
      If (feature 1 <= 2.9)
       Predict: 0.125
      Else (feature 1 > 2.9)
       Predict: 0.3392857142857143
    Else (feature 0 > 740.0)
     If (feature 0 <= 780.0)
      Predict: 1.0
     Else (feature 0 > 780.0)
      If (feature 1 <= 2.9)
       Predict: 0.0
      Else (feature 1 > 2.9)
       Predict: 1.0
   Else (feature 2 > 2.0)
    If (feature 0 <= 500.0)
     Predict: 0.0
    Else (feature 0 > 500.0)
     If (feature 1 <= 3.01)
      If (feature 0 <= 600.0)
       Predict: 0.2857142857142857
      Else (feature 0 > 600.0)
       Predict: 1.0
     Else (feature 1 > 3.01)
      If (feature 0 <= 540.0)
       Predict: 0.25
      Else (feature 0 > 540.

To see how well the model performed we create a column the indicates either accept (1) or reject (0).

In [12]:
import org.apache.spark.ml.feature.Binarizer

val binarizer: Binarizer = new Binarizer().
  setInputCol("prediction").
  setOutputCol("binarized_prediction").
  setThreshold(0.5)

val predictionBinary = binarizer.transform(predictions) 

predictionBinary.show

+-----+---+----+----+----------------+-----+-------------------+--------------------+
|admit|gre| gpa|rank|        features|label|         prediction|binarized_prediction|
+-----+---+----+----+----------------+-----+-------------------+--------------------+
|    0|340|2.92|   3|[340.0,2.92,3.0]|  0.0|                0.0|                 0.0|
|    0|340|3.15|   3|[340.0,3.15,3.0]|  0.0|                0.0|                 0.0|
|    0|360|3.14|   1|[360.0,3.14,1.0]|  0.0| 0.3392857142857143|                 0.0|
|    0|380|2.94|   3|[380.0,2.94,3.0]|  0.0|                0.0|                 0.0|
|    0|380|3.59|   4|[380.0,3.59,4.0]|  0.0|0.23076923076923078|                 0.0|
|    0|380|3.61|   3|[380.0,3.61,3.0]|  0.0|0.23076923076923078|                 0.0|
|    0|400|2.93|   3|[400.0,2.93,3.0]|  0.0|                0.0|                 0.0|
|    0|400|3.05|   2|[400.0,3.05,2.0]|  0.0| 0.3392857142857143|                 0.0|
|    0|400|3.08|   2|[400.0,3.08,2.0]|  0.0| 0.3392857

First we look to see the false positives and false negative. Filter the data to include only where the model differs from the actual action. As you can see there are some interesting cases. In my test set there is a student with800 GRE score, 3.75 GPA and from first rank school that was denied, but the model predict that they should be admitted. Seems clear that the model is correct in admitting that student.

In [16]:
import org.apache.spark.sql.functions._
val wrongPredictions = predictionBinary.where(expr("label != binarized_prediction"))
wrongPredictions.show

+-----+---+----+----+----------------+-----+-------------------+--------------------+
|admit|gre| gpa|rank|        features|label|         prediction|binarized_prediction|
+-----+---+----+----+----------------+-----+-------------------+--------------------+
|    0|460|3.44|   2|[460.0,3.44,2.0]|  0.0|                1.0|                 1.0|
|    0|480|3.45|   2|[480.0,3.45,2.0]|  0.0|                1.0|                 1.0|
|    0|540|3.51|   2|[540.0,3.51,2.0]|  0.0| 0.6666666666666666|                 1.0|
|    0|540|3.78|   4|[540.0,3.78,4.0]|  0.0| 0.6666666666666666|                 1.0|
|    0|560|3.59|   2|[560.0,3.59,2.0]|  0.0| 0.6666666666666666|                 1.0|
|    0|560| 4.0|   3| [560.0,4.0,3.0]|  0.0| 0.6666666666666666|                 1.0|
|    0|580| 4.0|   1| [580.0,4.0,1.0]|  0.0| 0.6666666666666666|                 1.0|
|    0|600|3.48|   2|[600.0,3.48,2.0]|  0.0|                1.0|                 1.0|
|    0|660|3.49|   2|[660.0,3.49,2.0]|  0.0|          

We can aggregate the data to see how often the model differs from what was actual done. In my run the model admits 12 students that were denied (false positives) and denies 19 students that were admitted (false negatives). 

In [18]:
val countErrors = wrongPredictions.groupBy("label").agg(count("prediction").alias("Errors"))
countErrors.show

+-----+------+
|label|Errors|
+-----+------+
|  0.0|    12|
|  1.0|    19|
+-----+------+



This table is missing some important information. If our test set has 1,000 students then being wrong on 31 students gives an error rate of 3.1%, which is rather good. If the test set has 100 students then the error rate is 31%, which is not so good. Further more assuming there were 1,000 students since we don't know how many were admitted and how many were denied we can not know how well the model performed. If the school was very selective 950 students could have been denied and 50 accepted. In that case the model did well predicting who to deny but was rather poor in predicting who to admit. So lets see how often the model agrees with the data. 

In [20]:
val correctPredictions = predictionBinary.where(expr("label == binarized_prediction"))
val countCorrectPredictions = correctPredictions.groupBy("label").agg(count("prediction").alias("Correct"))
countCorrectPredictions.show

+-----+-------+
|label|Correct|
+-----+-------+
|  0.0|     75|
|  1.0|     12|
+-----+-------+



We can combine the these two tables into what is called the **confusion matrix**. You can learn more about the confusion matrix at http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/

| n=118 | Predicted Deny | Predicted Admit |
|--- |---|---|
|Actual Deny| 75| 12 |
|Actual Admit| 19| 12|



As you can see the model did fairly well at predicting who to deny, but was poor at predicting who to admit. Of the 31 students admitted the model predicted 19 would be denied. Adding totals and labeling true positives (TP), true negatives (TN), false positives (FP) and false negatives (FN) we get

| n=118 | Predicted Deny | Predicted Admit | Total |
|--- |---|---|
|Actual Deny| TN=75| FP=12 | 87|
|Actual Admit| FN=19| TP=12|31
| Total | 94| 24||

Using this table we can compute the following commom metrics

**Accuracy** How often is the model correct 
+ (TP+TN)/total = (75+12)/118 = 0.74

**Misclassification Rate** How often is the model wrong
+ (FP+FN)/total = (19+12)/118 = 0.26

**True Positive Rate or Sensitivity or Recall** When actually admit (yes) how often does model predict admit (yes)
+ TP/actual Admit = 12/31 = 0.39  

**False Positive Rate** When it's actually no (deny), how often does it predict yes (admit)
+ FP/actual no = 12/87 = 0.14

**Specificity** When it's actually no, how often does it predict no
+ TN/actual = 75/87 = 0.86

**Precision** When it predicts yes, how often is it correct
+ TP/predicted yes = 12/24 = 0.5

**Prevalence** How often does the yes condition actually occur in our sample
+ actual yes/total = 31/118 = 0.26

If you run the code in this notebook your results will differ. The problem is that the confusion matrix was calculated by hand. That means each time the code is run you will have to recompute the confusion matrix by hand again and all the metrics above. Go ahead and rerun the code and then recompute by hand the confusion matrix by hand. I think you will come to realize that **you should do all calculations in a notebook using code**.

To that end how would you compute the confusion matrix?